In [1]:
!pip install Flask Flask-SQLAlchemy Flask-JWT-Extended


Defaulting to user installation because normal site-packages is not writeable


In [3]:
from flask import Flask, request, jsonify
from flask_sqlalchemy import SQLAlchemy
from flask_jwt_extended import JWTManager, create_access_token, jwt_required, get_jwt_identity
from werkzeug.security import generate_password_hash, check_password_hash


In [5]:
app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///test.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['JWT_SECRET_KEY'] = 'your_secret_key'  # Replace with your secret key
db = SQLAlchemy(app)
jwt = JWTManager(app)


In [7]:
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    password = db.Column(db.String(120), nullable=False)

class Item(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(80), nullable=False)
    description = db.Column(db.String(200), nullable=True)


In [9]:
with app.app_context():
    db.create_all()


In [11]:
@app.route('/register', methods=['POST'])
def register():
    data = request.get_json()
    hashed_password = generate_password_hash(data['password'], method='sha256')
    new_user = User(username=data['username'], password=hashed_password)
    db.session.add(new_user)
    db.session.commit()
    return jsonify({'message': 'User registered successfully!'})


In [13]:
@app.route('/login', methods=['POST'])
def login():
    data = request.get_json()
    user = User.query.filter_by(username=data['username']).first()
    if not user or not check_password_hash(user.password, data['password']):
        return jsonify({'message': 'Invalid credentials!'}), 401
    
    access_token = create_access_token(identity={'username': user.username})
    return jsonify({'access_token': access_token})


In [15]:
@app.route('/item', methods=['POST'])
@jwt_required()
def create_item():
    data = request.get_json()
    new_item = Item(name=data['name'], description=data.get('description', ''))
    db.session.add(new_item)
    db.session.commit()
    return jsonify({'message': 'Item created successfully!'})


In [17]:
@app.route('/items', methods=['GET'])
@jwt_required()
def get_items():
    items = Item.query.all()
    output = []
    for item in items:
        item_data = {'id': item.id, 'name': item.name, 'description': item.description}
        output.append(item_data)
    return jsonify({'items': output})


In [19]:
@app.route('/item/<int:id>', methods=['PUT'])
@jwt_required()
def update_item(id):
    data = request.get_json()
    item = Item.query.get(id)
    if not item:
        return jsonify({'message': 'Item not found!'}), 404

    item.name = data['name']
    item.description = data.get('description', item.description)
    db.session.commit()
    return jsonify({'message': 'Item updated successfully!'})


In [21]:
@app.route('/item/<int:id>', methods=['DELETE'])
@jwt_required()
def delete_item(id):
    item = Item.query.get(id)
    if not item:
        return jsonify({'message': 'Item not found!'}), 404
    
    db.session.delete(item)
    db.session.commit()
    return jsonify({'message': 'Item deleted successfully!'})


In [ ]:
from flask import Flask
from werkzeug.serving import run_simple

run_simple('localhost', 5000, app)


 * Running on http://localhost:5000
Press CTRL+C to quit
127.0.0.1 - - [27/Aug/2024 12:58:56] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [27/Aug/2024 12:58:56] "GET /favicon.ico HTTP/1.1" 404 -
